## model 按token输出

In [ ]:
from typing import Any
from vllm import LLM
from transformers import AutoTokenizer, TextStreamer
from langchain_community.llms import VLLM
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
import sys


def on_llm_new_token(self, token: str, **kwargs: Any) -> None:
    sys.stdout.write(token)
    sys.stdout.flush()


model_name = "/root/models/Qwen1.5-1.8B-Chat"
tokenizer = AutoTokenizer.from_pretrained(model_name)
streamer = TextStreamer(tokenizer)
llm_llama3 = VLLM(
    model=model_name,
    tokenizer=tokenizer,
    task="text-generation",
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
    temperature=0.2,
    do_sample=True,
    repetition_penalty=1.1,
    return_full_text=False,
    max_new_tokens=900,
)

In [ ]:
from threading import Thread
from transformers import TextIteratorStreamer
inputs = tokenizer(["An increasing sequence: one,"], return_tensors="pt")
streamer = TextIteratorStreamer(tokenizer)

# Run the generation in a separate thread, so that we can fetch the generated text in a non-blocking way.
# generation_kwargs = dict(inputs, streamer=streamer, max_new_tokens=20)
# thread = Thread(target=llm_llama3.generate, kwargs=generation_kwargs)
# thread.start()
# generated_text = ""
# for new_text in streamer:
#     generated_text += new_text
#     print(generated_text)
llm_llama3.generate(["nihao"], streamer=streamer, max_new_tokens=20)

## transfomer streaming test

In [ ]:
from torch import half
from transformers import AutoModelForCausalLM, AutoTokenizer, TextStreamer
from vllm import LLM, SamplingParams

model_name = "/root/models/Qwen1.5-1.8B-Chat"
tok = AutoTokenizer.from_pretrained(model_name)
# model = AutoModelForCausalLM.from_pretrained(model_name)

llm_llama3 = LLM(
    model=model_name,
    tokenizer=model_name,
    dtype=half,
    # temperature=0.2,
    # do_sample=True,
    # repetition_penalty=1.1,
    # return_full_text=False,
    # max_new_tokens=900,
)

# inputs = tok(["世界你好"], return_tensors="pt")
streamer = TextStreamer(tok)


# Despite returning the usual output, the streamer will also print the generated text to stdout.
# _ = llm_llama3.generate(**inputs, streamer=streamer, max_new_tokens=20)

In [ ]:

from vllm import SamplingParams
from transformers import TextStreamer


model_name = "/root/models/Qwen1.5-1.8B-Chat"
# llm_llama3 = LLM(
#     model=model_name,
#     tokenizer=model_name,
#     # temperature=0.2,
#     # do_sample=True,
#     # repetition_penalty=1.1,
#     # return_full_text=False,
#     # max_new_tokens=900,
# )

llm_llama3 = LLM(
    model=model_name,
    tokenizer=model_name,
)
sampling_params = SamplingParams(top_p=0.95,
                                 temperature=0.2,
                                 repetition_penalty=1.1,
                                 max_tokens=900,
                                 )
# streamer = TextStreamer(tok)
# sampling_params = SamplingParams(top_p=0.95,
#                                  temperature=0.2,
#                                  #  do_sample=True,
#                                  repetition_penalty=1.1,
#                                  max_tokens=900,

#                                  #  return_full_text=False,
#                                  #  max_new_tokens=900,
#                                  )


outputs = llm_llama3.generate(
    "其中 LLM类主要运行离线程序、SamplingParams负责设置调用模型当中的一些参数", sampling_params=sampling_params)

# llm_llama3("其中 LLM类主要运行离线程序、SamplingParams负责设置调用模型当中的一些参数")

# for output in outputs:
#     prompt = output.prompt
#     generated_text = output.outputs[0].text
#     print(f"Prompt: {prompt!r}, Generated text: {generated_text!r}")

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, TextStreamer

model_name = "/root/models/Qwen1.5-1.8B-Chat"
tok = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, temperature=0.2,
                                             do_sample=True,
                                             repetition_penalty=1.1,)
inputs = tok(["An increasing sequence: one,"], return_tensors="pt")
streamer = TextStreamer(tok)

# Despite returning the usual output, the streamer will also print the generated text to stdout.
_ = model.generate(**inputs, streamer=streamer,
                   max_new_tokens=200)

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, TextIteratorStreamer
from threading import Thread

model_name = "/root/models/Qwen1.5-1.8B-Chat"
tok = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)
# model = AutoModelForCausalLM.from_pretrained(llm_llama3)

inputs = tok(["An increasing sequence: one,"], return_tensors="pt")
streamer = TextIteratorStreamer(tok)

# Run the generation in a separate thread, so that we can fetch the generated text in a non-blocking way.
generation_kwargs = dict(inputs, streamer=streamer, max_new_tokens=20)
thread = Thread(target=model.generate, kwargs=generation_kwargs)
thread.start()
generated_text = ""
for new_text in streamer:
    generated_text += new_text
    print(generated_text)

## 根据文档生成问题示例